In [1]:
import numpy as np 
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df=pd.read_csv(r"D:\cleaned_dataset\metadata.csv")

In [3]:
df

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,discharge,[2010. 7. 21. 15. 0. ...,4,B0047,0,1,00001.csv,1.6743047446975208,NaN,NaN
1,impedance,[2010. 7. 21. 16. 53. ...,24,B0047,1,2,00002.csv,NaN,0.05605783343888099,0.20097016584458333
2,charge,[2010. 7. 21. 17. 25. ...,4,B0047,2,3,00003.csv,NaN,NaN,NaN
3,impedance,[2010 7 21 20 31 5],24,B0047,3,4,00004.csv,NaN,0.05319185850921101,0.16473399914864734
4,discharge,[2.0100e+03 7.0000e+00 2.1000e+01 2.1000e+01 2...,4,B0047,4,5,00005.csv,1.5243662105099023,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7560,impedance,[2010. 9. 30. 7. 36. ...,24,B0055,247,7561,07561.csv,NaN,0.0968087979207628,0.15489738203707232
7561,discharge,[2010. 9. 30. 8. 8. ...,4,B0055,248,7562,07562.csv,1.0201379996149256,NaN,NaN
7562,charge,[2010. 9. 30. 8. 48. 54.25],4,B0055,249,7563,07563.csv,NaN,NaN,NaN
7563,discharge,[2010. 9. 30. 11. 50. ...,4,B0055,250,7564,07564.csv,0.9907591663373165,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   type                 7565 non-null   object
 1   start_time           7565 non-null   object
 2   ambient_temperature  7565 non-null   int64 
 3   battery_id           7565 non-null   object
 4   test_id              7565 non-null   int64 
 5   uid                  7565 non-null   int64 
 6   filename             7565 non-null   object
 7   Capacity             2794 non-null   object
 8   Re                   1956 non-null   object
 9   Rct                  1956 non-null   object
dtypes: int64(3), object(7)
memory usage: 591.1+ KB


In [5]:
impedence_df=df[(df['type']=='impedance') & df[['Re','Rct']].notna().all(axis=1)]

In [6]:
impedence_df

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
1,impedance,[2010. 7. 21. 16. 53. ...,24,B0047,1,2,00002.csv,NaN,0.05605783343888099,0.20097016584458333
3,impedance,[2010 7 21 20 31 5],24,B0047,3,4,00004.csv,NaN,0.05319185850921101,0.16473399914864734
13,impedance,[2010. 7. 22. 17. 3. ...,24,B0047,13,14,00014.csv,NaN,0.05963791501051059,0.21039872263834902
15,impedance,[2010. 7. 22. 20. 40. 25.5],24,B0047,15,16,00016.csv,NaN,0.05512505361624278,0.1754882075917004
17,impedance,[2010. 7. 23. 11. 35. ...,24,B0047,17,18,00018.csv,NaN,0.058878485312444453,0.19095687096090014
...,...,...,...,...,...,...,...,...,...,...
7536,impedance,[2010. 9. 28. 16. 5. ...,24,B0055,223,7537,07537.csv,NaN,0.09747109901247247,0.15760912100719615
7546,impedance,[2010. 9. 29. 8. 16. ...,24,B0055,233,7547,07547.csv,NaN,0.0982550032520246,0.14877104776241656
7548,impedance,[2010. 9. 29. 11. 53. ...,24,B0055,235,7549,07549.csv,NaN,0.09778722154159296,0.15088524149658153
7558,impedance,[2010. 9. 30. 3. 59. ...,24,B0055,245,7559,07559.csv,NaN,0.09867936363638956,0.15274663175569908


In [11]:
import re
from datetime import datetime

def parse_start_time(entry):
    entry = re.sub(r"[^\d\s]", "", entry) 
    entry = re.sub(r"\s+", " ", entry).strip()

    components = entry.split()

    if len(components) < 3:
        return f"Error parsing: {entry}"

    
    while len(components) < 6:
        components.append("0")

    try:
       
        seconds = int(float(components[5])) % 60 

        
        parsed_time = datetime(
            int(components[0]),  
            int(components[1]),  
            int(components[2]),  
            int(components[3]), 
            int(components[4]),  
            seconds             
        )
        return parsed_time.strftime("%Y-%m-%d %H:%M:%S")
    except ValueError as e:
        return f"Error parsing: {entry} ({e})"



In [12]:
impedence_df.loc[:,'start_time']=impedence_df['start_time'].apply(parse_start_time)

In [13]:
impedence_df

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
1,impedance,2010-07-21 16:53:08,24,B0047,1,2,00002.csv,NaN,0.05605783343888099,0.20097016584458333
3,impedance,Error parsing: 20100721 203105,24,B0047,3,4,00004.csv,NaN,0.05319185850921101,0.16473399914864734
13,impedance,2010-07-22 17:03:42,24,B0047,13,14,00014.csv,NaN,0.05963791501051059,0.21039872263834902
15,impedance,2010-07-22 20:40:15,24,B0047,15,16,00016.csv,NaN,0.05512505361624278,0.1754882075917004
17,impedance,2010-07-23 11:35:35,24,B0047,17,18,00018.csv,NaN,0.058878485312444453,0.19095687096090014
...,...,...,...,...,...,...,...,...,...,...
7536,impedance,2010-09-28 16:05:23,24,B0055,223,7537,07537.csv,NaN,0.09747109901247247,0.15760912100719615
7546,impedance,2010-09-29 08:16:48,24,B0055,233,7547,07547.csv,NaN,0.0982550032520246,0.14877104776241656
7548,impedance,2010-09-29 11:53:23,24,B0055,235,7549,07549.csv,NaN,0.09778722154159296,0.15088524149658153
7558,impedance,2010-09-30 03:59:18,24,B0055,245,7559,07559.csv,NaN,0.09867936363638956,0.15274663175569908


In [20]:
impedence_df.iloc[1:2]

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
3,impedance,Error parsing: 20100721 203105,24,B0047,3,4,00004.csv,NaN,0.05319185850921101,0.16473399914864734


In [21]:
impedence_df = impedence_df[~impedence_df['start_time'].str.contains("Error parsing")]

impedence_df.reset_index(drop=True, inplace=True)

In [22]:
impedence_df

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,impedance,2010-07-21 16:53:08,24,B0047,1,2,00002.csv,NaN,0.05605783343888099,0.20097016584458333
1,impedance,2010-07-22 17:03:42,24,B0047,13,14,00014.csv,NaN,0.05963791501051059,0.21039872263834902
2,impedance,2010-07-22 20:40:15,24,B0047,15,16,00016.csv,NaN,0.05512505361624278,0.1754882075917004
3,impedance,2010-07-23 11:35:35,24,B0047,17,18,00018.csv,NaN,0.058878485312444453,0.19095687096090014
4,impedance,2010-07-23 15:12:58,24,B0047,19,20,00020.csv,NaN,0.05281282159808424,0.17225152334151322
...,...,...,...,...,...,...,...,...,...,...
1512,impedance,2010-09-28 16:05:23,24,B0055,223,7537,07537.csv,NaN,0.09747109901247247,0.15760912100719615
1513,impedance,2010-09-29 08:16:48,24,B0055,233,7547,07547.csv,NaN,0.0982550032520246,0.14877104776241656
1514,impedance,2010-09-29 11:53:23,24,B0055,235,7549,07549.csv,NaN,0.09778722154159296,0.15088524149658153
1515,impedance,2010-09-30 03:59:18,24,B0055,245,7559,07559.csv,NaN,0.09867936363638956,0.15274663175569908


In [26]:
import plotly.graph_objects as go
import plotly.io as pio

# Set renderer to work in Kaggle notebooks
pio.renderers.default = 'iframe_connected'
for battery_id in impedence_df['battery_id'].unique() :
    current_battery_info = impedence_df[impedence_df['battery_id'] == battery_id]
    # print(current_battery_info)
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=current_battery_info['start_time'], 
        y=current_battery_info['Re'], 
        mode='lines', 
        name='Re',
        line=dict(color='green')
    ))
    fig.add_trace(go.Scatter(
        x=current_battery_info['start_time'], 
        y=current_battery_info['Rct'], 
        mode='lines', 
        name='Rct',
        line=dict(color='blue')
    ))
    fig.update_layout(
        title=f"Resistance (Battery ID: {battery_id})",
        xaxis_title="Start Time",
        yaxis_title="Resistance (Ohms)",
        xaxis=dict(tickangle=45),
        legend_title="Legend",
        template='plotly'
    )
    fig.show()

In [36]:

battery_info = {}
for idx, row in impedence_df.iterrows() :
    battery_id = row['battery_id']
    filename = row['filename']
    start_time = row['start_time']
    filePath = f"D:/cleaned_dataset/metadata.csv/{filename}
    try:
        data = pd.read_csv(filePath)
        
        if 'Battery_impedance' in data.columns:
            impedance_values = data['Battery_impedance'].apply(lambda x: complex(x).real)
            avg_impedance = impedance_values.mean()
            if battery_id not in battery_info:
                battery_info[battery_id] = {'start_time': [], 'battery_impedance': []}
            battery_info[battery_id]['start_time'].append(start_time)
            battery_info[battery_id]['battery_impedance'].append(avg_impedance)
        else:
            print(f"'Battery_impedance' column not found in {filename}")
    except Exception as e:
        print(f"Error occurred with {filename}: {e}")
for battery_id, data in battery_info.items():
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data['start_time'],
                                 y=data['battery_impedance'],
                                 mode='lines',
                                 name=f'Battery {battery_id}')
                     )
        fig.update_layout(
            title=f'Battery impedance change over time for battery {battery_id}',
            xaxis_title='Start Time',
            yaxis_title='Battery Impedance (Ohms)',
            xaxis=dict(tickangle=45),
            template='plotly'
        )
        fig.show()

Error occurred with 00002.csv: [Errno 2] No such file or directory: 'D:/cleaned_dataset/metadata.csv/00002.csv'
Error occurred with 00014.csv: [Errno 2] No such file or directory: 'D:/cleaned_dataset/metadata.csv/00014.csv'
Error occurred with 00016.csv: [Errno 2] No such file or directory: 'D:/cleaned_dataset/metadata.csv/00016.csv'
Error occurred with 00018.csv: [Errno 2] No such file or directory: 'D:/cleaned_dataset/metadata.csv/00018.csv'
Error occurred with 00020.csv: [Errno 2] No such file or directory: 'D:/cleaned_dataset/metadata.csv/00020.csv'
Error occurred with 00030.csv: [Errno 2] No such file or directory: 'D:/cleaned_dataset/metadata.csv/00030.csv'
Error occurred with 00032.csv: [Errno 2] No such file or directory: 'D:/cleaned_dataset/metadata.csv/00032.csv'
Error occurred with 00034.csv: [Errno 2] No such file or directory: 'D:/cleaned_dataset/metadata.csv/00034.csv'
Error occurred with 00036.csv: [Errno 2] No such file or directory: 'D:/cleaned_dataset/metadata.csv/000